<a href="https://colab.research.google.com/github/LamuneGitHub/A001_Python_Test/blob/main/EDA_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----
----
# # EDA ( Exploratory Data Analysis )( 탐색적 데이터 분석)
## 정의
> : 탐색적 데이터 분석  
: 데이터의 특징을 파악하는것

## 목적
> : 본격적인 데이터 분석이나 모델링 전에 데이터의 특징을 파악하여 분석 계획을 세우기 위한 기초공사 작업  
  : 유의미한 business insight(업무 통찰)을 얻기 위함  
  : 시행착오 비용 최소화

## 수행 과정 및 방법
* Data Description 확인 (데이터 명세서 )  
  : 데이터 명세서를 읽어서 데이터의 특성을 이해 

* 시각적 분석
  * 엑셀등을 이용하여 직접 눈으로 특성을 분석

* 프로그램적 분석
  * 분석 메소드 활용
    * info()
    * shape
    * description()
    * isnull().sum()
    * duplicated()
    * 기타 프로그램코딩 활용
  * Feature Engineering 
  * 시각화

> 목표 (프로그램적 분석의)
  * 결측치 확인
    * 컬럼별 결측치 비율
  * 카테고리 값의 비율
  * 가능 범위를 벗어난 값 확인
  * 계산값이 틀린 값 확인
  * 이상한 문자가 들어간 값 확인
  * 중복값

* Feature Engineering (변수 가공)
* Business Insight (업무 통찰)



# # EDA 수행

## 프로그램 기초 셋팅

In [1]:
# import

%matplotlib inline

from IPython.display import display

import re
import math
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import random

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

#pandas에서 DataFrame을 요약해서 표시하지 않도록 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 줄 구분 표시 출력 
def print_line() :
  print ("\n------------------------------------------------")
def print_line_m() :
  print ("\n-----------------------")
def print_line_s() :
  print ("\n----------")


In [ ]:
# # 구글 드라이브 mount
# from google.colab import drive
# drive.mount('/content/drive');


## 작업 DataFrame 지정

In [5]:
# 데이터 로딩

# # 엑셀 파일 로딩
# df_001 = pd.read_excel("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/stocks/Travel.xlsx", sheet_name = "008770 ")
# # CSV 파일 로딩
# df_red = pd.read_csv( "https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv" , sep = ",")
# df_white = pd.read_csv( "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv" , sep = ";")

# df_titanic = sns.load_dataset("titanic")

# df_patients = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/patients_info.csv')
# df_insuline_test = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/insuline_test.csv')

# df_final_revised = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/final_revised.csv')
# df_movie = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/movie.csv')

# df_discussion_114 = pd.read_csv('https://raw.githubusercontent.com/LamuneGitHub/A001_Python_Test/main/data/Discussion_114.csv')



# df_data = pd.read_csv('서울시_기간별_시간평균_대기환경_정보_2020.03.csv', encoding='euc-kr', usecols=['측정일시', '측정소명', '오존(ppm)'])

# read_csv 파라미터
"""    ,sep=','
    ,encoding='cp949' , encoding='euc-kr' , encoding='ISO-8859-1'
    , usecols=['측정일시', '측정소명', '오존(ppm)']
    , header=0 
"""


df_tmp = pd.read_csv( "https://raw.githubusercontent.com/LamuneGitHub/AI15_Prj_01/main/vgames2.csv" , sep=","  , header=0  )



## 시각적 탐색


In [6]:
# 작업 DataFrame 지정
df_target = df_tmp

#맨위 
display( df_target.head(20) )
#마지막
display( df_target.tail(20) )
#무작위
display( df_target.sample(20) )

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
5,6,Power Pro Kun Pocket 8,DS,2005.0,Sports,Konami Digital Entertainment,0,0,0.14,0
6,7,JASF: Jane's Advanced Strike Fighters,PC,2011.0,Simulation,Deep Silver,0.02,0.01,0,0
7,8,Samurai Warriors: State of War,PSP,2005.0,Action,Tecmo Koei,0.07,0,0.11,0.01
8,9,Imabikisou,PS3,2007.0,Adventure,Sega,0,0,0.04,0
9,10,The Penguins of Madagascar: Dr. Blowhole Retur...,PS3,2011.0,Action,THQ,0.04,0.03,0,0.01


,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
16578,16579,Gex: Enter the Gecko,PS,1998.0,Platform,BMG Interactive Entertainment,320K,0.22,0,0.04
16579,16580,Super Ghouls 'n Ghosts,GBA,2002.0,Platform,Capcom,0.05,0.02,0,0
16580,16581,Captain Morgane and the Golden Turtle,DS,2012.0,Adventure,Reef Entertainment,0,0.03,0M,0
16581,16582,LEGO The Hobbit,XOne,2014.0,Action,Warner Bros. Interactive Entertainment,0.13,0.12,0,0.02
16582,16583,Shanghai: True Valor,PS,1998.0,Puzzle,Activision,0.01,0.01,0,0
16583,16584,LEGO Indiana Jones 2: The Adventure Continues,Wii,2009.0,Action,Activision,0.91,0.39,0,0.13
16584,16585,Bionicle Heroes,DS,2006.0,Shooter,Eidos Interactive,0.06,0,0,0
16585,16586,Pokemon Channel,GC,2003.0,Adventure,Nintendo,0.24,0.06,0.07,0.01
16586,16587,Jikkyou Powerful Pro Yakyuu 2000 Ketteiban,PS,2000.0,Sports,Konami Digital Entertainment,0,0,0.08,0.01
16587,16588,Need for Speed Underground 2,XB,2004.0,NaN,Electronic Arts,1.38,0.8,0,0.1


,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
14158,14159,Summon Night 2,DS,2008.0,Role-Playing,Namco Bandai Games,0,0,0.06,0
10826,10827,ShellShock 2: Blood Trails,X360,2009.0,Shooter,Eidos Interactive,0.05,0.03,0M,0.01
753,754,Pursuit Force: Extreme Justice,PSP,2007.0,Action,Sony Computer Entertainment,0.07,0.07,0,0.03
9474,9475,Mortal Kombat: Shaolin Monks,PS2,2005.0,Action,Midway Games,0.58,0.02,0,0.09
2348,2349,Skydiving Extreme,PS,2001.0,Sports,Banpresto,0.04,0.03,0,0.01
4881,4882,Street Fighter II': Special Champion Edition (...,GEN,1993.0,Action,Capcom,0,0,0.07,0M
7455,7456,Rollcage Stage II,PS,2000.0,Racing,Sony Computer Entertainment,0.05,0.03,0,0.01
15045,15046,Persona Q: Shadow of the Labyrinth,3DS,2014.0,Role-Playing,Nippon Ichi Software,0.33,0.07,0.27,0.04
2705,2706,All-Star Baseball 2003,PS2,2002.0,Sports,Acclaim Entertainment,0.38,0.29,0,0.1
6231,6232,Golden Sun: Dark Dawn,DS,2010.0,Role-Playing,Nintendo,0.56,0.11,0.13,0.05


----
----
## 프로그램적 분석 ( 자동 분석 프로그램 )


In [ ]:
# 데이터셋 정보 확인
#: 컬럼 자료형 , 전체 자료 갯수 , 컬럼 갯수 , 결측치 여부 , 전체 data 크기
print_line()
print ( "# 데이터셋 정보 확인" )
print ( df_target.info() )

# 컬럼별 통계치 확인
print_line()
print ( "#컬럼별 통계특성 확인" )
print ( df_target.describe())


# 컬럼 자료형 조회 (dtypes 이용) 
#df_titanic.dtypes

# 데이터의 행열 크기 조회
print_line()
print( "# 데이터의 행열 크기 = " , df_target.shape )

# 결측치가 존재하는 컬럼명 조회
print_line()
print( "# 컬럼별 결측치 갯수 ")
print( df_target.isnull().sum() )

print_line()
print( "# 결측치 총 갯수 = " , df_target.isnull().sum().sum())


# 각 컬럼별 값의 비율 조회
# 컬럼명 조회
print_line()
print( "# 컬럼별 값의 비율 ")
list_columns = df_target.columns
print ("컬럼 목록= " , list_columns)

# 컬럼별 값의 비율 조회
for tmp_column in list_columns :
  df_temp = df_target[tmp_column].value_counts(normalize=True ,dropna = True)
  
  print_line_s()
  print ( "컬럼명 = " , tmp_column )
  
  if len(df_temp) <= 20 :
    print ( df_temp )
  else :
    print ( "값의 종류가 20개 이상임" )

# 컬럼별 포함된 큭수문자 의 종류 조회
print_line()
print( "# 컬럼별 포함된 특수문자 종류 ")
regx_특수문자제외한 = r'([^0-9a-zA-Z]+)'
for tmp_column in list_columns :

  if df_target[tmp_column].dtype == 'object'  :
    print ( "컬럼명 = " , tmp_column )
    
    df_rslt = df_target[tmp_column].str.extractall( r'([^0-9a-zA-Z]+)'  )
    #display ( type(df_rslt) ) #=> DataFrame 

    set_rslt = []
    for idx in df_target.index :
      
      # TODO 값이 Nan인 경우의 정규표현식을 조사하지 않도록 한다,
      # 우선은 str인 경우만 
      tmp_val = df_target.loc[ idx , tmp_column]
      if isinstance( tmp_val , str) : 
        rslt = re.findall ( regx_특수문자제외한 , df_target.loc[ idx , tmp_column] )
        set_rslt += rslt 
      #else :
      #  display ( tmp_val )

    print( set(set_rslt) )
    
    print_line_s()


# 컬럼별 중복값 확인
print_line()
print( "# 컬럼별 중복값 확인 ")
for tmp_column in list_columns :
  
  # 중복값 갯수 조회
  tmp_dup_count = df_target[tmp_column].duplicated(keep = False).sum()
  
  if tmp_dup_count > 0 :
    print ( tmp_column , "컬럼에 중복값", tmp_dup_count , "개" )
    #print ( "index = " , df_target[tmp_column].duplicated(keep = False)   )
    #TODO : 중복값 인덱스 출력
  
# 중복 데이터 확인
print( "중복 data 갯수" , df_target.duplicated().sum() )


#TODO 컬럼별 값 분포 그래프 표시
#Boxplot과 Histogram을 통해서도 이상치 존재 여부를 확인해볼 수 있습니다.





In [ ]:
# 중복 데이터 여부 확인
# keep = False : 동일한 중복값이 3개 인 경우 False를 지정해 주지 않은 경우 기본설정은 첫번째 중복값은 중복이 아닌것으로 취급되므로 
#                중복데이터 갯수가 2개가 나오게 되므로 
#                중복값이 3개로 반환 될 수 있도록 False 설정을 해준다.
df_target.duplicated(keep = False).sum()

In [ ]:
# 중복된 항목을 제거한 이후의 data 갯수 조회
len(df_target.drop_duplicates('email'))

In [ ]:
df_target = df_titanic

# 사망자그룹의 비율 계산
# value_counts(normalize=True) => 비율 
df_temp = df_target['survived'].value_counts(normalize=True ,dropna = True)
df_temp

In [ ]:
# 반올림 
#: ( 소숫점 2째자리까지 반올림 예제)
print (df_temp[0].round(2))

In [ ]:
# 사망자 그룹의 숫자 확인
# value_counts()
df_temp = df_titanic['survived'].value_counts(dropna = True) # 갯수 구하기
df_temp

In [ ]:
# 사망자 그룹의 비율 확인
# ,dropna = True : Nan 값도 포함 할지 여부 
df_titanic['survived'].value_counts(normalize=True ,dropna = True)

## Feature Engineering
: 수학적인 연산과 도메인 지식을 이용하여 raw data로부터 유용한 feature를 도출해 내는 과정

In [ ]:
## 범위에 따라 단계 분류한 값을 age_class 컬럼을 생성하여 입력 
df_temp = df_titanic

bins = [-2, 0, 19, 59, 999]
labels = ['unknown','young', 'middle', 'old']
df_temp['age_class'] = pd.cut(x=df_temp['age'], bins=bins, labels=labels)
df_temp.head(3)

### 각 테이블의 공통된 컬럼명 조회

In [ ]:
# 테이블별 중복 column name 확인
all_columns = pd.Series(list(df_patients) + list(df_insuline_test) + list(df_side_effects))
all_columns
all_columns.duplicated() # 중복여부 배열 
all_columns[all_columns.duplicated()]

### 테이블 합치기 (수직)

In [ ]:
# (TODO) 테이블 합치기 (수직)

# column의 이름 수정
df_red.rename(columns = {'total_sulfur-dioxide' : 'total_sulfur_dioxide'}, inplace=True)

# 합치려는 데이터셋의 columns가 동일한지 확인
set(df_red.columns) == set(df_white.columns)

# 하나의 데이터셋으로 통합
df = df_white.append(df_red, ignore_index = True)

# 잘 합쳐졌는지 확인
df.shape

In [ ]:

# (TODO) 테이블 합치기 (수퍙)

# (TODO) 컬럼 삭제
# (TODO) 컬럼값 정제
# (TODO) 컬럼 추가
red_color = np.repeat('red', len(df_red))
df_red['color'] = red_color
df_red.head()



## 시각화


### 한글 폰트 설치
: 설치 후 메뉴>런타임>런타임 다시 시작 할 것

In [ ]:
# 한글 폰트 설치 
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 폰트 셋팅
plt.style.use('ggplot')
font = {'size': 12,
        'family': 'NanumBarunGothic'}
mpl.rc('font', **font)

----

In [ ]:
# 생존여부, 성별 별 평균요금 df 조회
df_temp = df_titanic.groupby( ['survived' ,'sex'] ).mean()
df_temp

In [ ]:
df_temp

In [ ]:
df_red['color'] = np.repeat('red', len(df_red))
df_white['color'] = np.repeat('white', len(df_white))
df_temp = df_red.append(df_white)

df_temp.groupby('color')['quality'].mean().plot(kind='bar', 
                                                title='Average Wine Quality by Color', 
                                                fontsize=13, 
                                                color=['red', 'blue'], 
                                                alpha=0.4);

In [ ]:
counts = df_temp.groupby(['quality', 'color']).count()['pH']
counts.plot(kind='bar', color=['red','blue'], title='Counts by Wine Quality and Color', fontsize=15, alpha=0.4);

In [ ]:
df_temp = df_titanic

# 막대그래프 표시 
df_temp.plot(kind = 'bar')

In [ ]:
df_temp = df_titanic

temp = df_temp.groupby(['sex', 'survived'])['fare'].mean().reset_index()
fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(data=temp, x='survived', y='fare', hue='sex', ax=ax)
ax.set_title("생존 여부와 성별에 따른 평균 요금")
ax.set_xticklabels(['Not Survived', 'Survived'])
plt.show();

In [ ]:
df_temp = df_titanic

fig, ax = plt.subplots(figsize=(8, 8))
fig = plt.pie(df_temp['survived'].value_counts(), labels=['No', 'Yes'], autopct='%.2f%%',
              textprops={'fontsize':14})
plt.title('Survived', fontsize=16)
plt.show()

In [ ]:
df_temp = df_titanic

fig, ax = plt.subplots(figsize=(8, 6))
sns.countplot(x=df_temp['pclass'].values, ax=ax)
plt.title('클래스별 수', fontsize=16)
plt.show()

In [ ]:
sns.countplot(x=df_temp['pclass'].values)

In [ ]:
df_temp['age'].plot(kind='hist', bins=8)

In [ ]:
df_temp = df_titanic

bins = np.arange(0, df_temp.age.max()+df_temp.age.max()/8, df_temp.age.max()/8)
fig, ax = plt.subplots(figsize=(8, 6))
fig = plt.hist(x=df_temp['age'], bins=bins)
plt.title('나이 분포', fontsize=16)
plt.xlabel('나이')
plt.ylabel('수')
plt.show()

In [ ]:
df_temp = df_red

# quality 에 따른 휘발성산의 차이를 scatter plot 으로 확인
plt.scatter(df_temp['quality'], df_temp['volatile acidity'])

In [ ]:
df_temp = df_red

# quality 에 따른 평균 휘발성산 확인
group_volatile_acidity = df_temp['volatile acidity'].groupby(df_temp['quality'])
print(group_volatile_acidity.mean())

# plot으로 시각화
plt.plot(group_volatile_acidity.mean());

----
# # Data Wrangling (데이터 정제) , Data Preprocessing (데이터 전처리)  
  : 실제 작업의 70~80%가 소요 된다고 함 

  * 데이터 품질
    * 결측값 처리
    * 잘못된 값 처리
  * 구조적 품질
    * 정규화



In [ ]:
# DataFrame 복사
df_patients_clean = df_patients.copy()

### Missing value (결측값)

특정 컬럼의 결측값 갯수 확인

In [ ]:
# 환자 데이터 중 키, 몸무게, 체질량지수 컬럼의 결측값 확인
df_patients_error_tmp = df_patients[df_patients['키'].isnull() | df_patients['몸무게'].isnull() | df_patients['체질량지수'].isnull()]
df_patients_error_tmp.head(3)

(TODO) 특정 컬럼의 결측값을 수정


### Outliers (잘못된 값,이상치)  수정


In [ ]:
# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()


In [ ]:
# DataFrame의 특정 컬럼의 data중에서 cond_sel 조건에 맞는 data를 fix_func 으로 수정 후 수정된 DataFrame을 반환
def fix_column_err_value ( df_temp_clean: pd.DataFrame , target_column: '수정할 column 명' 
                          , cond_sel:'수정 데이터 check 조건' 
                          , fix_func:'err를 수정할 함수' 
                          ) -> pd.DataFrame : #'수정된 DataFrame' 

  # err 조건에 해당하는 data 조회
  sri_err = df_temp_clean[ target_column ].apply( cond_sel )

  # 조회 결과 확인 
  display ( sri_err.sum() )
  display ( df_temp_clean.loc[ sri_err ] )


  # 오류를 수정 후 data 확인
  sri_fixed_col = df_temp_clean.loc[ sri_err ][[target_column]].apply( fix_func ,axis=1)
  #display (sri_fixed_col.head(3))

  # 오류 수정한 데이터를 전처리 작업용 DataFrame에 반영
  df_temp_clean.loc[ sri_err, target_column ] = sri_fixed_col

  # 반영 결과 data 확인 
  display ( df_temp_clean[sri_err])

  # 반영 결과 집계 확인 
  sri_err = df_temp_clean[ target_column ].apply( cond_sel )
  display ( sri_err.sum() )

  return df_temp_clean


* URL 주소 
  : https:// 로 시작 하지 않는 데이터의 앞에 https://를 붙인다. 

In [ ]:
# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'Poster_Link'
cond_err = lambda x: True if not x.startswith('https://') else False
fix_func = lambda x: 'https://' + x

# 수정 함수 호출
df_result = fix_column_err_value( df_temp_clean , target_column , cond_err, fix_func ) 



다른 열의 값을 가지고 와서 합치기

In [ ]:
# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()

# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'Series_Title'
cond_err = lambda x: True 

# case 01 
fix_func = lambda x: df_temp_clean.loc[x.name,"Runtime"] + df_temp_clean.loc[x.name,"Genre"]

# case 02 : 조건 체크 columm 과 변경 column이 다른 경우
def fix_func2 ( x ) :
  df_temp_clean.loc[x.name,"Certificate"] = df_temp_clean.loc[x.name,"Runtime"] + df_temp_clean.loc[x.name,"Genre"]
  return df_temp_clean.loc[x.name,target_column]

df_result = fix_column_err_value( df_temp_clean , target_column , cond_err, fix_func2 ) 




특정 컬럼을 필요 없는 문자 삭제 후 int로 타입 변경 



In [ ]:
# case 01

# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()

df_temp_clean['Runtime'] = df_temp_clean['Runtime'].str.extract('([0-9]+)').astype('int16')
# movie['Runtime'] = movie['Runtime'].str.split().str[0].astype('int16')

In [ ]:
# case 02

# 전처리 작업용 임시 DataFrame 생성
df_temp_clean = df_final_revised.copy()


# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'Runtime'
cond_sel = lambda x: True
fix_func = lambda x: re.search( r'([0-9]+)' , x).group(0)

# 처리할 값에 대한 예외 처리가 필요한 경우 조건 처리를 포함한 function을 사용하여야 한다. 
def fix_func2( input ):
    if not bool( input ) : 
      return np.nan
    if not isinstance( input ,str ) :
      print( type(input)  ,  input)
      return np.nan
    return re.search( r'([0-9]+)' , input ).group(0)

# 수정 함수 호출
df_result = fix_column_err_value( df_temp_clean , target_column , cond_sel, fix_func ) 
df_result.head()

# Lecture 에서 따온 특수문자 체크 코드

In [ ]:
reg_con_01 = r'[_,!,.,@#$%()^\d+]'

# 특수 문자가 포함되었는지 여부 체크
def has_errors(inputString):
  return bool(re.search( reg_con_01 , inputString))


# 이름에 섞여 있는 오류 데이터를 확인
error_condition = df_patients_clean.이름.apply(has_errors)
error_names = df_patients_clean[error_condition]
print(error_names)

print("-------------")
# 코드가 정확하게 오류들을 제거하는지 확인 
error_names.이름 = df_patients_clean.이름.str.replace(reg_con_01, '')
print(error_names)

print("-------------")
# 반영
df_patients_clean.이름 = df_patients_clean.이름.str.replace(reg_con_01, '')

# 반영결과 확인
df_patients_clean[error_condition] 

### 이메일 형식 체크

In [ ]:
df_discussion_114.head(20)

In [ ]:
regx_이메일 = r'[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*@[0-9a-zA-Z]([-_\.]?[0-9a-zA-Z])*\.[a-zA-Z]{2,3}'
regx_핸드폰 = r'\d{3}-\d{3,4}-\d{4}'

# 작업 DataFrame 지정
df_temp_clean = df_discussion_114.copy()

# 오류 검사 조건, 수정 함수, 대상 컬럼 명 
target_column = 'email'

#cond_sel = lambda x: False if re.search( regx_이메일 , x).group(0) else True
def cond_sel (x) :
  if isinstance ( x , str ) :
   if re.match( regx_이메일 , x) :
    return False
   else :
    return True
  else :
    # str이 아닌 경우= Nan 인 경우 , 수정 대상으로 체크
    return True

def fix_func(x) :
  # 수정 대상인 경우 (TODO) 알맞은 코딩을 해주어야 함
  return "(TODO) Fix"


# 수정 함수 호출
df_result = fix_column_err_value( df_temp_clean , target_column , cond_sel, fix_func ) 

# 수정 결과 확인
display(df_result.head(11))
display(df_result.tail(5))

### (TODO) 전화번호 형식 체크

### 컬럼의 분리

In [ ]:
# 환자나이 컬럼 분리
# 나이 컬럼에서 숫자만 뽑아내어 int로 변환 후 환자나이 컬럼을 생성 
df_patients_clean['환자나이'] = df_patients_clean.나이.str.extract('(\d+)').astype('int') 

# 성별 컬럼 분리
# 나이 컬럼에서 숫자를 모두 제거 후 성별 컬럼을 생성
df_patients_clean['성별'] = df_patients_clean.나이.str.replace(r'[^a-zA-Z]', '', regex=True) #숫자만 삭제 한다.

# 기존 나이 컬럼 제거
patients_clean = df_patients_clean.drop('나이', axis='columns' )


In [ ]:
# 성별을 category 데이터 타입으로 변화 , 영어를 한글로 변환
patients_clean.성별 = patients_clean.성별.replace({'male':'남', 'female':'여'})
patients_clean.성별 = patients_clean.성별.astype('category')

## 구조적 문제
df1.melt(id_vars='index', value_vars=['A', 'B'])

pd.melt(df1, id_vars = 'index', var_name='variable', value_name='value')

# 결과 데이터를 파일로 저장


In [ ]:
df.to_csv('winequality_edited.csv', index=False)